In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from pandas import Series, DataFrame
from ipywidgets import FloatProgress
from time import sleep
from IPython.display import display
import re
import pickle
#from urlextract import URLExtract
import json
import sqlite3
import datetime
from tqdm import tqdm

In [2]:
headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'}

# Womens

## Clothings

In [36]:
links=[
    ['https://www.ninecolours.com/sarees?page=',310,"sarees"],
    ['https://www.ninecolours.com/suits?page=',85,"suits"],
    ['https://www.ninecolours.com/lehengas?page=',62,'lehengas'],
    ['https://www.ninecolours.com/kurti?page=',75,'kurti'],
    ['https://www.ninecolours.com/gowns?page=',17,'gowns'],
    ['https://www.ninecolours.com/quirk-kurti?page=',6,'quirk_kurti']
]

In [37]:
prod_links=[]
for i in links:
    for j in tqdm(range(1,i[1])):
        try:
            result=requests.get(i[0]+str(j),headers=headers)
            get=result.content
            soup=BeautifulSoup(get,'lxml')
            prod=soup.find_all("div",{'class':'item_thumb'})
            for k in prod:
                try:
                    li=[]
                    li.append(k.find('a').attrs['href'])
                    li.append(i[2])
                    prod_links.append(li)
                except:
                    pass
        except:
            pass

100%|██████████| 5/5 [00:12<00:00,  2.59s/it]


In [42]:
Product_Link=[]
Product_Name=[]
Product_Brand=[]
Size_Available=[]
Price=[]
MRP=[]
Gender=[]
Description=[]
Cat_list=[]
Primary_Image_Link=[]
Secondary_Image_Link=[]

In [43]:
err=[]
for i in tqdm(prod_links):
    try:
        result = requests.get(i[0],headers=headers)
        get = result.content
        soup = BeautifulSoup(get,"lxml")

        product_name=soup.find('div',{'class':'product_column details_boxD'}).h1.text
        product_brand="nine colours"
        product_link=i[0]
        product_cat=i[1]
        product_des=soup.find('div',{'class':'description'}).text.lstrip().rstrip()

        product_price=int(''.join(soup.find('div',{'class':'updated-price'}).text[2:].split(',')))
        if(soup.find('div',{'class':'price-old'})):
            product_mrp=int(''.join(soup.find('div',{'class':'price-old'}).text[2:].split(',')))
        else:
            product_mrp=product_price
        if(soup.find('div',{'class':'product_column column_1 product_image_block'})):
            img=soup.find('div',{'class':'product_column column_1 product_image_block'}).find_all('li',{'class':'als-item'})
            images=[]
            for j in img:
                images.append(j.find('a').attrs['data-image'])
            primary_link=images[0]
            secondary_link=",".join(images[1:])
        else:
            primary_link=soup.find('meta',{'property':'twitter:image'}).attrs['content']
            secondary_link="Not Available"

        size_avl="One-Size"
    
    except:
        err.append(i[0])
        pass
    
    else:
        Product_Link.append(product_link)
        Product_Brand.append(product_brand)
        Product_Name.append(product_name)
        Description.append(product_des)
        Gender.append('Women')
        Cat_list.append(product_cat)
        MRP.append(product_mrp)
        Price.append(product_price)
        Primary_Image_Link.append(primary_link)
        Secondary_Image_Link.append(secondary_link)
        Size_Available.append(size_avl)

100%|██████████| 100/100 [04:30<00:00,  2.70s/it]


In [44]:
con=sqlite3.connect("ninecolors.db")  
cur=con.cursor()
cur.execute('CREATE TABLE product_details (Website varchar(40) not null,Product_Link text PRIMARY KEY,Product_Name varchar(50) not null,Product_Brand varchar(50) not null,Product_Category varchar(50),Size_Avail varchar(20) not null,Price int not null,MRP int not null,Gender varchar(15) not null,Description text not null,Primary_Image_Links text not null,Secondary_Image_Links text not null,Affiliate_Link text not null )')
for i in range(len(Product_Link)):
    try:
        cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)'%('https://www.ninecolours.com/',Product_Link[i],Product_Name[i],Product_Brand[i],Cat_list[i],Size_Available[i],Price[i],MRP[i],Gender[i],Description[i],Primary_Image_Link[i],Secondary_Image_Link[i],''))
    except:
        pass
con.commit()
con.close()

## Jewellery

In [5]:
links=[]
for i in tqdm(range(1,71)):
    try:
        result=requests.get(f"https://www.ninecolours.com/Jewellery?page={i}",headers=headers)
        get=result.content
        soup=BeautifulSoup(get,'lxml')
        prod=soup.find_all("div",{'class':'item_thumb'})
        for i in prod:
            try:
                links.append(i.find('a').attrs['href'])
            except:
                pass
    except:
        pass

100%|██████████| 70/70 [04:08<00:00,  3.54s/it]


In [48]:
Product_Link=[]
Product_Name=[]
Product_Brand=[]
Size_Available=[]
Price=[]
MRP=[]
Gender=[]
Description=[]
Cat_list=[]
Primary_Image_Link=[]
Secondary_Image_Link=[]

In [49]:
err=[]
for i in tqdm(links):
    try:
        result = requests.get(i,headers=headers)
        get = result.content
        soup = BeautifulSoup(get,"lxml")

        product_name=soup.find('div',{'class':'product_column details_boxD'}).h1.text
        product_brand="nine colours"
        product_link=i
        product_cat='Jewellery'
        product_des=soup.find('div',{'class':'description'}).text.lstrip().rstrip()

        product_price=int(''.join(soup.find('div',{'class':'updated-price'}).text[2:].split(',')))
        product_mrp=int(''.join(soup.find('div',{'class':'price-old'}).text[2:].split(',')))
        if(soup.find('div',{'class':'product_column column_1 product_image_block'})):
            img=soup.find('div',{'class':'product_column column_1 product_image_block'}).find_all('li',{'class':'als-item'})
            images=[]
            for j in img:
                images.append(j.find('a').attrs['data-image'])
            primary_link=images[0]
            secondary_link=",".join(images[1:])
        else:
            primary_link=soup.find('meta',{'property':'twitter:image'}).attrs['content']
            secondary_link="Not Available"

        size_avl="One-Size"
    
    except:
        err.append(i)
        pass
    
    else:
        Product_Link.append(product_link)
        Product_Brand.append(product_brand)
        Product_Name.append(product_name)
        Description.append(product_des)
        Gender.append('Women')
        Cat_list.append(product_cat)
        MRP.append(product_mrp)
        Price.append(product_price)
        Primary_Image_Link.append(primary_link)
        Secondary_Image_Link.append(secondary_link)
        Size_Available.append(size_avl)

100%|██████████| 100/100 [04:29<00:00,  2.70s/it]


In [53]:
con=sqlite3.connect("ninecolors.db")  
cur=con.cursor()
#cur.execute('CREATE TABLE product_details (Website varchar(40) not null,Product_Link text PRIMARY KEY,Product_Name varchar(50) not null,Product_Brand varchar(50) not null,Product_Category varchar(50),Size_Avail varchar(20) not null,Price int not null,MRP int not null,Gender varchar(15) not null,Description text not null,Primary_Image_Links text not null,Secondary_Image_Links text not null,Affiliate_Link text not null )')
for i in range(len(Product_Link)):
    try:
        cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)'%('https://www.ninecolours.com/',Product_Link[i],Product_Name[i],Product_Brand[i],Cat_list[i],Size_Available[i],Price[i],MRP[i],Gender[i],Description[i],Primary_Image_Link[i],Secondary_Image_Link[i],''))
    except:
        pass
con.commit()
con.close()

# Mens

In [3]:
links=[]
for i in tqdm(range(1,96)):
    try:
        result=requests.get(f"https://www.ninecolours.com/mens?page={i}",headers=headers)
        get=result.content
        soup=BeautifulSoup(get,'lxml')
        prod=soup.find_all("div",{'class':'item_thumb'})
        for i in prod:
            try:
                links.append(i.find('a').attrs['href'])
            except:
                pass
    except:
        pass

100%|██████████| 95/95 [05:28<00:00,  3.45s/it]


In [31]:
Product_Link=[]
Product_Name=[]
Product_Brand=[]
Size_Available=[]
Price=[]
MRP=[]
Gender=[]
Description=[]
Cat_list=[]
Primary_Image_Link=[]
Secondary_Image_Link=[]

In [32]:
err=[]
for i in tqdm(links):
    try:
        result = requests.get(i,headers=headers)
        get = result.content
        soup = BeautifulSoup(get,"lxml")

        product_name=soup.find('div',{'class':'product_column details_boxD'}).h1.text
        product_brand="nine colours"
        product_link=i
        product_cat='Ethnic Wear'
        product_des=soup.find('div',{'class':'description'}).text.lstrip().rstrip()

        product_price=int(''.join(soup.find('div',{'class':'updated-price'}).text[2:].split(',')))
        if(soup.find('div',{'class':'price-old'})):
            product_mrp=int(''.join(soup.find('div',{'class':'price-old'}).text[2:].split(',')))
        else:
            product_mrp=product_price
        if(soup.find('div',{'class':'product_column column_1 product_image_block'})):
            img=soup.find('div',{'class':'product_column column_1 product_image_block'}).find_all('li',{'class':'als-item'})
            images=[]
            for j in img:
                images.append(j.find('a').attrs['data-image'])
            primary_link=images[0]
            secondary_link=",".join(images[1:])
        else:
            primary_link=soup.find('meta',{'property':'twitter:image'}).attrs['content']
            secondary_link="Not Available"
        if(soup.find('select',{'class':'required_field std_require_on_cart'})):
            sizes=soup.find('select',{'class':'required_field std_require_on_cart'}).find_all('option')[1:]
            size=[]
            for i in sizes:
                size.append(i.text[:-1])
            size_avl=','.join(size)
        elif(soup.find("select",{'class':'update_option_price'})):
            sizes=soup.find('select',{'class':'update_option_price'}).find_all('option')[1:]
            size=[]
            for i in sizes:
                size.append(i.text[:-1])
            size_avl=','.join(size)
        else:
            size_avl="One-size"
            
            
    
    except:
        err.append(i)
        pass
    
    else:
        Product_Link.append(product_link)
        Product_Brand.append(product_brand)
        Product_Name.append(product_name)
        Description.append(product_des)
        Gender.append('Men')
        Cat_list.append(product_cat)
        MRP.append(product_mrp)
        Price.append(product_price)
        Primary_Image_Link.append(primary_link)
        Secondary_Image_Link.append(secondary_link)
        Size_Available.append(size_avl)

100%|██████████| 100/100 [04:28<00:00,  2.69s/it]


In [34]:
con=sqlite3.connect("ninecolors.db")  
cur=con.cursor()
#cur.execute('CREATE TABLE product_details (Website varchar(40) not null,Product_Link text PRIMARY KEY,Product_Name varchar(50) not null,Product_Brand varchar(50) not null,Product_Category varchar(50),Size_Avail varchar(20) not null,Price int not null,MRP int not null,Gender varchar(15) not null,Description text not null,Primary_Image_Links text not null,Secondary_Image_Links text not null,Affiliate_Link text not null )')
for i in range(len(Product_Link)):
    try:
        cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)'%('https://www.ninecolours.com/',Product_Link[i],Product_Name[i],Product_Brand[i],Cat_list[i],Size_Available[i],Price[i],MRP[i],Gender[i],Description[i],Primary_Image_Link[i],Secondary_Image_Link[i],''))
    except:
        pass
con.commit()
con.close()